<a href="https://colab.research.google.com/github/SudhanshuDubey5001/Twitter_sentiment_analysis_ML/blob/main/Sentiment_Analysis_using_Twitter_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install kaggle

In [ ]:
# configure path of Kaggle JSON
! mkdir -p ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d kazanova/sentiment140

 96% 78.0M/80.9M [00:00<00:00, 110MB/s] 
100% 80.9M/80.9M [00:00<00:00, 87.9MB/s]


In [ ]:
# Extract the zip file containing the dataset
from zipfile import ZipFile
dataset = '/content/sentiment140.zip'
with ZipFile(dataset, 'r') as zip:
  zip.extractall()
  print("Extraction complete!")

Extraction complete!


In [ ]:
# Import the dependencies
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer # to convert textual data to numerical
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
print(stopwords.words("english")) #these words don't have any inferential meaning and its not required for processing

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [ ]:
# Load datasets to panda dataframe
twitter_dataset = pd.read_csv('/content/training.1600000.processed.noemoticon.csv', encoding = 'ISO-8859-1')

In [ ]:
# check rows and columns of twitter dataset
twitter_dataset.shape

(1599999, 6)

In [ ]:
twitter_dataset.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [ ]:
# its reading the first row as column names. We need to explicitly tell the column names
column_names = ['target', 'id', 'date', 'flag', 'user', 'text']
twitter_dataset = pd.read_csv(
    '/content/training.1600000.processed.noemoticon.csv',
    names = column_names,
    encoding = 'ISO-8859-1'
    )

In [ ]:
twitter_dataset.shape

(1600000, 6)

In [ ]:
twitter_dataset.head()

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [ ]:
twitter_dataset.isnull().sum() # checking if there are null values

target    0
id        0
date      0
flag      0
user      0
text      0
dtype: int64

In [ ]:
#check the value distribution of target column
twitter_dataset['target'].value_counts()

0    800000
4    800000
Name: target, dtype: int64

In [ ]:
# since it is an equal distributuin, we don't need to normalize the dataset
# lets replace the 4 to 1 to make it more understandable. 0 -> negative tweet 1 -> positive tweet
twitter_dataset.replace({'target':{4:1}}, inplace = True)

In [ ]:
twitter_dataset['target'].value_counts()

0    800000
1    800000
Name: target, dtype: int64

Stemming -> reducing the word to its root word
Example: 'acting', 'actor', 'actoress' -> Root word: 'act'

In [ ]:
#lets do stemming of each tweet -
port_stem = PorterStemmer()
def stemming(content):
  stemmed_content = re.sub('[^a-zA-Z]', ' ', content)
  stemmed_content = stemmed_content.lower()
  stemmed_content = stemmed_content.split()
  stemmed_content = [
                      port_stem.stem(word)
                      for word in stemmed_content
                        if not word in stopwords.words('english')
                    ]
  return ' '.join(stemmed_content)


In [ ]:
c = stemming("you are my sunshine and only my sunshine, you made me happy when skies are grey")
print(c)

sunshin sunshin made happi sky grey


In [ ]:
# lets apply the stemming to our twitter_dataset by creating a new column for that
# and putting all the stemmed data in it
twitter_dataset['stemmed_data'] = twitter_dataset['text'].apply(stemming)

In [ ]:
twitter_dataset.head()

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [23]:
X = twitter_dataset['stemmed_data'].values
Y = twitter_dataset['target'].values

In [24]:
print(X)

['switchfoot http twitpic com zl awww bummer shoulda got david carr third day'
 'upset updat facebook text might cri result school today also blah'
 'kenichan dive mani time ball manag save rest go bound' ...
 'readi mojo makeov ask detail'
 'happi th birthday boo alll time tupac amaru shakur'
 'happi charitytuesday thenspcc sparkschar speakinguph h']


In [25]:
print(Y)

[0 0 0 ... 1 1 1]


Let's split the data into training data and test data.

Training data: We use the training data to train the ML model about tweets and the relationships between target and text. Larger the training set, better.

Test data: Test data is used to check if the model is performing well on unseen data and avoid overfitting which happens when the model memorizes the model too well and gets biased with the answers.

In [27]:
[X_train, X_test, Y_train, Y_test] = train_test_split(
    X,
    Y,
    stratify = Y,
    test_size = 0.2,  # this means we split the test data as 20% of entire data set, hence 80% is reserved for the training data
    random_state = 2 # this defines the splitting of data in a manner. Each number provides different way of splitting data
    )

In [28]:
print(X.shape, X_train.shape, X_test.shape) # this is how the dataset is splitted

(1600000,) (1280000,) (320000,)


Now, lets convert the textual dataset into numerical values so that our ML model can understand the data. We use TfidVectorizer.

- Takes the data and build a vocabulary based on the different words encountered
- Find the Term Frequency (TF) of each word meaning, how many times the word is repeated in the document among the dataset. TF = no. of times word appear/total no. of words in document. This gives how important the word is in that document.
- Find the Inverse Document frequency (IDF) of each word in the dataset. This value is used to downnplay the importance of words that appear frequently in all documents and emphasize the importance of words that appear rarely across all documents but are profound in one document.
IDF = log(total no. of docs/no. of docs containing the word)
- Then we find out the weight of each word by -
weight = TF * IDF
- And create a document-matrix where the rows are all the documents and columns are all the words that appear with their weights in each cell.    

In [29]:
document_vector = TfidfVectorizer()
# fit : learn the parameters of the transformer which is used to convert the data into TF-IDF weight matrix.
# parameters include mean and standard deviation of the data and IDF values.
X_train = document_vector.fit_transform(X_train)
# we only use transform for test data because we already applied fit_transform to training set and therefore
# the ML already learnt the parameters of the traning data. It can apply the same process to test.
X_test = document_vector.transform(X_test)

In [30]:
print(X_train)

  (0, 443066)	0.4484755317023172
  (0, 235045)	0.41996827700291095
  (0, 109306)	0.3753708587402299
  (0, 185193)	0.5277679060576009
  (0, 354543)	0.3588091611460021
  (0, 436713)	0.27259876264838384
  (1, 160636)	1.0
  (2, 288470)	0.16786949597862733
  (2, 132311)	0.2028971570399794
  (2, 150715)	0.18803850583207948
  (2, 178061)	0.1619010109445149
  (2, 409143)	0.15169282335109835
  (2, 266729)	0.24123230668976975
  (2, 443430)	0.3348599670252845
  (2, 77929)	0.31284080750346344
  (2, 433560)	0.3296595898028565
  (2, 406399)	0.32105459490875526
  (2, 129411)	0.29074192727957143
  (2, 407301)	0.18709338684973031
  (2, 124484)	0.1892155960801415
  (2, 109306)	0.4591176413728317
  (3, 172421)	0.37464146922154384
  (3, 411528)	0.27089772444087873
  (3, 388626)	0.3940776331458846
  (3, 56476)	0.5200465453608686
  :	:
  (1279996, 390130)	0.22064742191076112
  (1279996, 434014)	0.2718945052332447
  (1279996, 318303)	0.21254698865277746
  (1279996, 237899)	0.2236567560099234
  (1279996, 2910

In [31]:
print(X_test)

  (0, 420984)	0.17915624523539803
  (0, 409143)	0.31430470598079707
  (0, 398906)	0.3491043873264267
  (0, 388348)	0.21985076072061738
  (0, 279082)	0.1782518010910344
  (0, 271016)	0.4535662391658828
  (0, 171378)	0.2805816206356073
  (0, 138164)	0.23688292264071403
  (0, 132364)	0.25525488955578596
  (0, 106069)	0.3655545001090455
  (0, 67828)	0.26800375270827315
  (0, 31168)	0.16247724180521766
  (0, 15110)	0.1719352837797837
  (1, 366203)	0.24595562404108307
  (1, 348135)	0.4739279595416274
  (1, 256777)	0.28751585696559306
  (1, 217562)	0.40288153995289894
  (1, 145393)	0.575262969264869
  (1, 15110)	0.211037449588008
  (1, 6463)	0.30733520460524466
  (2, 400621)	0.4317732461913093
  (2, 256834)	0.2564939661498776
  (2, 183312)	0.5892069252021465
  (2, 89448)	0.36340369428387626
  (2, 34401)	0.37916255084357414
  :	:
  (319994, 123278)	0.4530341382559843
  (319995, 444934)	0.3211092817599261
  (319995, 420984)	0.22631428606830145
  (319995, 416257)	0.23816465111736276
  (319995, 3

Now we will train the ML model based on our dataset.

We will use Logistic Regression to train the model

In [34]:
model = LogisticRegression(max_iter=1000) #we got through the model 1000 times so we get a good accuracy
# now we will train the data where LogisticRegression will understand the relationship between tweets and target 0 or 1
model.fit(X_train, Y_train)

LogisticRegression(max_iter=1000)

The "model" instance is now trained by training data. Now we can check the accuracy of the model

Accuracy Check

In [37]:
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(Y_train, X_train_prediction)
print('Accuracy Score training data : ', training_data_accuracy * 100, '%')

Accuracy Score training data :  81.018984375 %


Since, we trained the data using Y_train, we also need to check with the unseen data i,e X_test and Y_test

In [38]:
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(Y_test, X_test_prediction)
print('Accuracy score test data: ', test_data_accuracy*100, '%')

Accuracy score test data:  77.80375000000001 %


The training data is like preparing for exam and then test data serves like an exam where there are unseen questions on the same concept. The accuracy score of test data is pretty good and close to the training data accuracy. This means there is no overfitting condition.  

Now let's save the model using library called pickle.

In [39]:
import pickle

In [40]:
filename = 'twitter_sentiment_analysis_trained_model.sav'
pickle.dump(model, open(filename, 'wb'))  #wb means writing in binary format

Lets check if the model works for individual tweets

In [48]:
loaded_model = pickle.load(open('/content/twitter_sentiment_analysis_trained_model.sav','rb'))
tweetPos = 80
X_single_tweet = X_test[tweetPos]
prediction = loaded_model.predict(X_single_tweet)
if(prediction[0] == 0):
  print('Predicted tweet sentiment: ', 'Negative tweet')
else:
  print('Predicted tweet sentiment: ', 'Positive tweet')
print('Actual tweet sentiment: ', Y_test[tweetPos])

  (0, 457255)	0.746057801237267
  (0, 340999)	0.2625248713404701
  (0, 338619)	0.3433519940901109
  (0, 230883)	0.3274683479672717
  (0, 157529)	0.33496562619992915
  (0, 145988)	0.19274430637272733
Predicted tweet sentiment:  Positive tweet
Actual tweet sentiment:  1
